In [1]:
# Imports for scraping individual websites
import json
import re
from bs4 import BeautifulSoup
from collections import defaultdict


In [33]:
def extract_prefix(full_string, prefix, sentences_removed=1):
    if full_string.startswith(prefix):
        # closing_parenthesis_index = full_string.find("-")  # Find the position of the first closing parenthesis
        # if closing_parenthesis_index != -1:
        #     extracted_text = full_string[:closing_parenthesis_index + 1]  # Include the closing parenthesis
        #     return extracted_text
        
    # return full_string
        # Split the string by periods
        split_strings = full_string.split('.')
        
        # Remove the first two split strings
        split_strings = split_strings[sentences_removed:]
        
        # Rejoin the remaining split strings with periods
        result_string = '.'.join(split_strings)
        
        return result_string.strip()
    else:
        return full_string


def extract_sentences(string, no_sentences=1):
    return '.'.join(string.split('.')[no_sentences:]).strip()


In [34]:
def scrape_soup(soup):
    """
    Scrapes and process text from a webpage, focusing on <p> tags.

    Parameters:
    url (str): The URL of the webpage to scrape.

    Returns:
    str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
    """
    
    # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
    def get_class(p):
        return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"
    
    p_dict = defaultdict(list)
    try:
        for element in soup.find_all():
            if element.name == "header" or element.name == "style" or element.name == "blockquote" or element.name == "strong":
                element.decompose()
            elif element.attrs and 'class' in element.attrs:
                if any("fl-node-aou15irpt4cb" in class_name.lower() 
                        or "podcast__wrapper" in class_name.lower()
                    for class_name in element['class']):
                        # print("slice-author-bio found")
                        for sibling in element.find_all_next():
                            sibling.decompose()
                        element.decompose()
                        break
                elif any("caption" in class_name.lower()
                    for class_name in element['class']):
                    element.decompose()
            elif element.text and (element.text.startswith("Cast:")):
                for sibling in element.find_all_next():
                    sibling.decompose()
                element.decompose()
                break
              
            # elif element.name == "hr" or (element.text and element.text.startswith("Director:")):
            #     # Perform actions on the element with the specific id
            #     for sibling in element.find_all_next():
            #         sibling.decompose()
            #     element.decompose()
                # break

            # elif element.name == "footer" or element.name == "h4":
            #     for sibling in element.find_all_next():
            #         sibling.decompose()
            #     element.decompose()

                        
        # finds all <p> tags
        paragraphs = soup.find_all('p')
        for p in paragraphs:
            if len(p.text.split(' ')) > 5 and not p.find(['ol', 'ul', 'dl' ]) and not p.text.startswith("Director"):
                tags= p.find_all(['i', 'em']) 
                if len(tags) == 1 and tags[0].text == p.text:
                    continue
                if p.find('a'):
                    # Check if the anchor text is the same as the paragraph text
                    if p.find('a').text == p.text:
                        continue 
                p_dict[get_class(p)].append(p.text)
        # print(p_dict)
        # Find the class with the longest list of paragraphs
        longest_key = max(p_dict, key=lambda k: len(p_dict[k]))
        # Combine and clean text
        full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()
        full_text = full_text[:full_text.rfind(".")]

    except Exception as e:
        print("Exception "  + str(e) + " occured")
        return ""

    return full_text


In [35]:
def process_json(json_obj):
    fullText = scrape_soup(BeautifulSoup(json_obj['fullText'], "html.parser"))
    json_obj['fullText'] = fullText
    json_obj["spanAttribution"] = [{"authorID":json_obj['authorIDs'][0],
                                    "start":0,
                                    "end":len(fullText)}]
    json_obj["lengthWords"] = len(fullText.split(' '))
    return json_obj


In [2]:
jsonl_file_path = '/shared/3/projects/hiatus/rotten_tomatoes/raw_output/reviewsoups.jsonl'
rt_corpus_path = '../output/reviews.jsonl'
rt_corpus_path = '/shared/3/projects/hiatus/rotten_tomatoes/raw_data/rtcorpus.jsonl'



In [37]:
# with open(jsonl_file_path, 'r') as jsonl_file, \
#      open(rt_corpus_path, 'a') as corpus:
#     seen_text = set()
#     for line in jsonl_file:
#         json_obj = json.loads(line.strip())
#         # print(json_obj['fullText'])
#         # break
#         processed_json = process_json(json_obj)
#         if processed_json['lengthWords'] >= 50 \
#             and processed_json["fullText"] not in seen_text:
#             seen_text.add(processed_json['fullText'])
#             corpus.write(json.dumps(processed_json) + '\n')
#             # break


Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg is an empty sequence occured
Exception max() arg 

In [3]:
# gets the number of unique authors
# Read the JSONL file and store JSON objects in a list
with open(rt_corpus_path, 'r') as file:
    json_list = [json.loads(line) for line in file]
    # print(json_list)

    # Create a set to store unique "fullText" values
    unique_ids = defaultdict(int)

    # Create a new list to store filtered JSON objects
    filtered_json_list = []

    # # Iterate through the JSON objects
    for json_obj in json_list:
        # Extract the "fullText" value
        
        ids = json_obj.get("authorIDs")
        unique_ids[ids[0]]+=1
    
    count = 0
    for id in unique_ids:
        if(unique_ids[id] > 1):
            count += 1

    print(count)




798
